In [29]:
import pandas as pd
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sample_submission=pd.read_csv('/content/sample_submission.csv')

In [23]:
from sklearn.model_selection import train_test_split

train , val = train_test_split( train , train_size = 0.80,                               # Iternations , size = 0.80, 0.70, 0.60
                               stratify = train['target'],
                               random_state = 42)


In [30]:
X_train = train.drop('target',axis = 1)
y_train = train['target']

X_val = val.drop('target',axis = 1)
y_val = val['target']

KeyError: ignored

In [43]:
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model_RF = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy = 'mean'), 
    StandardScaler(),
    RandomForestClassifier(random_state = 91, n_estimators = 340 , max_depth = 10)
    )
#fit on train
model_RF.fit(X_train, y_train)


#score on train, val 
print('Train Accuracy', model_RF.score(X_train, y_train))
print('Val Accuracy', model_RF.score(X_val, y_val))


Train Accuracy 0.9951690821256038
Val Accuracy 1.0


In [45]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

param_distributions = {    
    'simpleimputer__strategy': ['mean', 'median','mode'], 

    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    
}

model_RF_2 = GridSearchCV(
    model_RF, 
    param_grid =param_distributions, 
    n_jobs =3, 
    cv=None, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
)

model_RF_2.fit(X_train, y_train);
print('Train Accuracy', model_RF_2.score(X_train, y_train))
print('Val Accuracy', model_RF_2.score(X_val, y_val))

y_pred_baseline=model_RF_2.predict_proba(test)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.9/dist-packages

Train Accuracy 0.8478260869565217
Val Accuracy 0.8554216867469879


In [48]:
y_pred_baseline

,0,1
0,0.678423,0.321577
1,0.442321,0.557679
2,0.235434,0.764566
3,0.453463,0.546537
4,0.622570,0.377430
...,...,...
271,0.258595,0.741405
272,0.803859,0.196141
273,0.381139,0.618861
274,0.731437,0.268563


In [47]:
submission_baseline = sample_submission[['id']].copy()
y_pred_baseline=pd.DataFrame(y_pred_baseline)
submission_baseline=y_pred_baseline[1]
submission_baseline.to_csv('Baseline.csv' , index = False)

In [34]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

#set up a pipeline 

model_KNN = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy = 'median'), 
    StandardScaler(),
    KNeighborsClassifier(n_neighbors = 15, metric = 'minkowski')
    )
#fit on train
model_KNN.fit(X_train, y_train)


#score on train, val 
print('Train Accuracy', model_KNN.score(X_train, y_train))
print('Val Accuracy', model_KNN.score(X_val, y_val))


#predict on test 


Train Accuracy 0.7463768115942029
Val Accuracy 0.7590361445783133


In [40]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from scipy.stats import randint, uniform

param_distributions = {    
    'simpleimputer__strategy': ['mean', 'median','mode'], 

    'kneighborsclassifier__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
}

model_KNN_2 = GridSearchCV(
    model_KNN, 
    param_grid =param_distributions, 
    n_jobs =3, 
    cv=None, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
)

model_KNN_2.fit(X_train, y_train);
print('Train Accuracy', model_KNN_2.score(X_train, y_train))
print('Val Accuracy', model_KNN_2.score(X_val, y_val))

y_pred_KNN = model_KNN_2.predict(test) 


Fitting 5 folds for each of 21 candidates, totalling 105 fits
Train Accuracy 0.7681159420289855
Val Accuracy 0.7710843373493976


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 105.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.9/dist-package

In [38]:
submission_baseline = sample_submission[['id']].copy()
submission_baseline['target'] = y_pred_KNN
submission_baseline.to_csv('Baseline.csv' , index = False)

In [13]:
!pip install --upgrade category_encoders



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 7.8 MB/s eta 0:00:00


In [6]:
train.columns

Index(['id', 'gravity', 'ph', 'osmo', 'cond', 'urea', 'calc', 'target'], dtype='object')